In [1]:
import os
import sys
import time

import numpy as np
import pygame
from gymnasium import Env
from gymnasium.spaces import Discrete, Dict, Box, MultiDiscrete, Tuple

from Agents.agent import Agent
# from Agents.fov_points import get_fov_points
# from Agents.overlap_detection import detect_overlapping_points
from Agents.RayCast import get_fov_rays
from Constants.constants import WHITE, RED, BLUE, SCREEN_WIDTH, SCREEN_HEIGHT, WALLS, FOV_RADIUS, LEVEL_1_WALLS, LEVEL_2_WALLS, LEVEL_3_WALLS
from Walls.collision_detection import detect_collision
from Walls.wall_class import Walls

sys.path.insert(1, os.path.join(sys.path[0], '..'))

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [42]:
WALLS2 = LEVEL_2_WALLS
class GameEnv(Env):
    metadata = {"render_modes" : ["human", "rgb_array"], "render_fps": 4}

    def __init__(self, render_mode=None):
        super(GameEnv, self).__init__()

        # defining the screen dimension for render purpose
        self.screen_width = SCREEN_WIDTH
        self.screen_height = SCREEN_HEIGHT

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode


        # total_values = 219
        # self.observation_space = Box(low=np.zeros(total_values, dtype=np.float32), 
        #                             high=self.screen_width * np.ones(total_values, dtype=np.float32), 
        #                             dtype=np.float32)

        self.observation_space = Dict({
            "predator_position": Box(low=np.array([0, 0], dtype=np.float32),
                                    high=np.array([self.screen_width, self.screen_height], dtype=np.float32),
                                    dtype=np.float32),
            "predator_angle": Discrete(360),
            "destination_coordinates": Box(low=np.array([0, 0, 0, 0], dtype=np.float32),
                                    high=np.array([self.screen_width, self.screen_height, self.screen_width, self.screen_height], dtype=np.float32),
                                    dtype=np.float32),
        })
        # defining the observation and action spaces for all the agents
        # self.observation_space = None

        # defining the action space based on total number of predator and prey
        # since we are training only one agent so, defining only the necessary number of actions
        self.action_space = Discrete(3)
        # 5 for rotate
        # clockwise, anti-clock
        # move front, move back and wait

        self.total_steps = 0

        self.number_of_predator = 1

        self.predator_agent = None

        self.predator_total_reward = 0

        self.obs = None

        # start the tick timer
        self.start_time = 0
        self.total_running_time = 10

        # the pygame window should be initialized in the render function
        # initializing the pygame
        # pygame.init()

        # # setting the screen size
        # self.screen = pygame.display.set_mode((self.screen_width, self.screen_height))
        # pygame.display.set_caption('Multi Agent Environment(simple)')

        # # initializing the font
        # pygame.font.init()
        # self.font = pygame.font.Font(None, 18)
        self.window = None
        self.clock = None

        # for the wall initializations
        self.wall = Walls(pygame)
        self.walls = None

    def agent_init(self):
        predator_agents = Agent('predator', 0)

        self.predator_agent = predator_agents

    def flatten_list(self, nested_list):
        flattened_list = []
        for item in nested_list:
            if isinstance(item, list) :
                flattened_list.extend(self.flatten_list(item))
            else:
                flattened_list.append(item)
        return flattened_list
    
    def _get_obs(self):
        top = 0
        bottom = 0
        if self.predator_agent.current_position[0] < self.walls[0].midbottom[0]:
            top = np.array(self.walls[0].bottomright, dtype=np.float32)
            bottom = np.array(self.walls[1].topright, dtype=np.float32)
        
        if self.walls[0].right < self.predator_agent.current_position[0] < self.walls[2].midbottom[0]:
            top = np.array(self.walls[2].bottomright, dtype=np.float32)
            bottom = np.array(self.walls[3].topright, dtype=np.float32)
        
        # print(f'top:{top}, buttom:{bottom}')

        observation = {
            "predator_position": self.predator_agent.current_position,
            "predator_angle": self.predator_agent.angle,
            "destination_coordinates": np.concatenate([top, bottom]),
        }

        # print(observation)
        return observation
    
    # to capture all the info
    def _get_info(self):
        direction = 0

        if self.predator_agent.current_position[0] < self.walls[0].midbottom[0]:
            mid_point = (np.array(self.walls[0].midbottom, dtype=np.float32)  + np.array(self.walls[1].midtop, dtype=np.float32)) / 2
            # print(f'mid point: {mid_point}')
            direction = mid_point - self.predator_agent.current_position
        
        if self.walls[0].right < self.predator_agent.current_position[0] < self.walls[2].midbottom[0]:
            mid_point = (np.array(self.walls[2].midbottom, dtype=np.float32) + np.array(self.walls[3].midtop, dtype=np.float32)) / 2
            direction = mid_point - self.predator_agent.current_position
        
        # here goes a proximal reward function to maximize any trial of getting close to the goal
        distance = np.linalg.norm(direction)

        info = {
            "distance": distance,
        }

        return info


    def _max_right(self):
        max_right = 0

        for wall in self.walls:
            if wall.right > max_right:
                max_right = wall.right
        
        return max_right

    def get_reward(self, reward):
        reward = reward
        direction = 0
        crossed = False


        if self.predator_agent.current_position[0] < self.walls[0].midbottom[0]:
            mid_point = (np.array(self.walls[0].midbottom, dtype=np.float32) + np.array(self.walls[1].midtop, dtype=np.float32)) / 2
            direction = mid_point - self.predator_agent.current_position

        if self.walls[0].right < self.predator_agent.current_position[0] < self.walls[2].midbottom[0]:
            mid_point = (np.array(self.walls[2].midbottom, dtype=np.float32) + np.array(self.walls[3].midtop, dtype=np.float32)) / 2
            direction = mid_point - self.predator_agent.current_position
            crossed = True
    

        distance = np.linalg.norm(direction)
        if distance == 0:
            distance = 1
            
        reward += 1/distance

        if crossed and self.predator_agent.current_position[0] < self.walls[0].right:
            reward -= 0.01

        print(f'direction: {direction}, distance:{distance}, reward: {reward}')
        return reward


    # the usual reset function
    def reset(self, seed=None, option=None):
        super().reset(seed=seed)
        self.start_time = time.time()

        self.agent_init()
        self.wall.clear_walls()
        self.walls = self.wall.make_wall(WALLS2)

        # self.set_obs_space()

        self.total_steps = 0
        self.predator_total_reward = 0

        predator = self.predator_agent

        # for predator in self.predator_agents:
        predator.agent_reset(width=self.screen_width, height=self.screen_height, walls=self.walls)
        # observation.append([predator.index, predator.agent, predator.current_position])

        # setting the predator and prey to their initial position

        self.predator_agent = predator


        # all the variable values inside the observation space needs to be sent inside the observation variable
        # for this level purpose we decided to add the dictionary observation
        # set the observation to a dictionary
        observation = self._get_obs()
        self.obs = observation
        info = self._get_info()

        return observation, info

    def step(self, action):
        # initializing the return variables
        done = False
        reward = 0
        truncated = False
        info = {}
        current_time = time.time()

        elapsed_time = current_time - self.start_time

        self.predator_agent.step_update(action, range_x=self.screen_width, range_y=self.screen_height)
        self.predator_agent = detect_collision(self.predator_agent, self.walls)

        # observation needs to be set a dictionary

        self.total_steps += 1
        reward = self.get_reward(reward)
        

        if self.predator_agent.current_position[0] > self.walls[1].right:
            reward += 100
        # for wall in self.walls:
        if self.predator_agent.current_position[0] > self._max_right():
            reward += 200
            done = True

        if elapsed_time >= self.total_running_time + 10:
            reward -= 100
            done = True
        """
        here lies the most important task
        handling the rewards
        """
        
        # getting observation and info
        observation = self._get_obs()
        info = self._get_info()

        self.predator_total_reward = reward
        self.obs = observation

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, done, truncated, info

    def render(self):
        if self.render_mode == 'rgb_array':
            self._render_frame()


    def _render_frame(self):
        if self.window is None and self.render_mode == "human":
            pygame.init()
            pygame.display.init()
            self.window = pygame.display.set_mode((self.screen_width, self.screen_height))
            pygame.font.init()

        if self.clock is None and self.render_mode == "human":
            self.clock = pygame.time.Clock()

        screen = pygame.Surface((self.screen_width, self.screen_height))
        screen.fill(WHITE)
        
        predator = self.predator_agent
        pygame.draw.circle(screen, RED, predator.center, predator.radius)
        pygame.draw.line(screen, RED, predator.center, predator.draw_direction_end, 5)

        mid_point = 0

        if self.predator_agent.current_position[0] < self.walls[0].midbottom[0]:
            mid_point = (np.array(self.walls[0].midbottom, dtype=np.float32)  + np.array(self.walls[1].midtop, dtype=np.float32)) / 2
            # print(f'mid point: {mid_point}')
            direction = mid_point - self.predator_agent.current_position
        
        if self.walls[0].right < self.predator_agent.current_position[0] < self.walls[2].midbottom[0]:
            mid_point = (np.array(self.walls[2].midbottom, dtype=np.float32) + np.array(self.walls[3].midtop, dtype=np.float32)) / 2
            direction = mid_point - self.predator_agent.current_position
        mid_point = (int(mid_point[0]), int(mid_point[1]))
        pygame.draw.line(screen, RED, predator.center, mid_point, 2)

        for key, wall in WALLS2.items():
            pygame.draw.rect(screen, BLUE, (wall['x'], wall['y'], wall['width'], wall['height']))

        if self.render_mode == "human":

            font = pygame.font.Font(None, 18)
        
            text_surface = font.render(f"Reward: {self.predator_total_reward: .5f} ", True, (0, 0, 0))

            text_rect = text_surface.get_rect()

            text_rect.center = (self.screen_width - 200, 10)
            
            screen.blit(text_surface, text_rect)
            self.window.blit(screen, screen.get_rect())
            pygame.event.pump()
            pygame.display.update()

            # this part is to fix the fps of rendering
            # self.clock.tick(self.metadata["render_fps"])
        
        else:
            return np.transpose(
                np.array(pygame.surfarray.pixels3d(screen)), axes=(1, 0, 2)
            )

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.font.quit()
            pygame.quit()

In [43]:
env = GameEnv(render_mode="human")

In [44]:
res = env.reset()

In [49]:
# env.reset()
env.predator_agent.current_position = np.array([30, 300], dtype=np.float32)
done = False
traunc = False
while not traunc:
    action = env.action_space.sample()
    _, _, done, traunc, _ = env.step(action)


direction: [170. 200.], distance:262.48809814453125, reward: 0.003809696542695737
direction: [170. 200.], distance:262.48809814453125, reward: 0.003809696542695737
direction: [170. 200.], distance:262.48809814453125, reward: 0.003809696542695737
direction: [170. 200.], distance:262.48809814453125, reward: 0.003809696542695737
direction: [170. 200.], distance:262.48809814453125, reward: 0.003809696542695737
direction: [170. 200.], distance:262.48809814453125, reward: 0.003809696542695737
direction: [170. 200.], distance:262.48809814453125, reward: 0.003809696542695737
direction: [170. 200.], distance:262.48809814453125, reward: 0.003809696542695737
direction: [170.17364827 200.98480774], distance:263.3514068814265, reward: 0.0037972077379113765
direction: [170.34729654 201.96961547], distance:264.21568275341514, reward: 0.0037847866923678072
direction: [170.34729654 201.96961547], distance:264.21568275341514, reward: 0.0037847866923678072
direction: [170.68931684 202.90930804], distance

KeyboardInterrupt: 

In [8]:
env.close()

In [9]:
from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [10]:
log_path = os.path.join('Training', 'Logs', 'Level_02_DQN')
baseline_path = os.path.join('Training', 'Models', 'Level_02_DQN')
callback_basealine_path = os.path.join('Training', 'Models', 'callback_Level_02_DQN')

In [16]:
load_path = os.path.join('Training', 'Models', 'callback_Level_01_DQN', 'best_model')

In [13]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=202, verbose=1)
eval_callback = EvalCallback(env, 
                                callback_on_new_best=stop_callback, 
                                eval_freq=100000, 
                                best_model_save_path=baseline_path, 
                                verbose=1)

In [31]:
model = DQN('MultiInputPolicy', env, verbose=1, tensorboard_log=log_path )

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [32]:
env.reset()
model.learn(total_timesteps=7000000, callback=eval_callback)

Logging to Training\Logs\Level_02_DQN\DQN_6
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.78e+03 |
|    ep_rew_mean      | -71.4    |
|    exploration_rate | 0.969    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 288      |
|    time_elapsed     | 80       |
|    total_timesteps  | 23120    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.58e+03 |
|    ep_rew_mean      | -76.6    |
|    exploration_rate | 0.939    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 279      |
|    time_elapsed     | 160      |
|    total_timesteps  | 44676    |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 5.33e+03 |
|    ep_rew_mean      | -73.1    |
|    exploration_rate | 0.913    |
| time/    

KeyboardInterrupt: 

In [104]:
model.save(baseline_path)

In [105]:
del model

In [92]:
from stable_baselines3.common.evaluation import evaluate_policy

In [106]:
model = DQN.load(baseline_path)

In [107]:
env = GameEnv(render_mode='human')

In [122]:
evaluate_policy(model, env, n_eval_episodes=10)

(206.93253619526513, 0.0)

In [21]:
env.close()

In [ ]:
%tensorboard